# ZEPTO EDA ANALYSIS
### The dataset takes the data from `https://www.kaggle.com/datasets/palvinder2006/zepto-inventory-dataset`
### Though as per online the dataset varies from the price given, still the EDA has been performed

`zepto_v2.csv`

## Importing all the libraries needed for the project
### Used libraries are pandas, plotly, subplots, graph_objs, and rich
1. pandas - dataset to dataframe construction, reading and manipulation
2. plotly - for multiple plots includeing histogram, pie and 3d-scatter
3. subplots - for multiple subplots of the pie charts
4. graph_objs - construction of pie plots for subplot generation
5. rich - better console text visualization

In [1]:
import pandas as pd, numpy as np, plotly.express as px
from rich.console import Console
from plotly.subplots import make_subplots as mks
import plotly.graph_objs as go
cons = Console()

### Reading library and Custom Color Palete

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/SoultySprings/zepto-inv-dataset/master/EDA/zepto_v2.csv', encoding='cp1252')
colors = ["#4E79A7", "#F28E2B", "#E15759", "#76B7B2", "#59A14F", "#EDC949", "#AF7AA1", "#FF9DA7", "#9C755F", "#BAB0AC", "#1F77B4", "#FF7F0E", "#2CA02C", "#D62728", "#9467BD"]

### Identify min and max quantities across the dataframe


In [3]:
print(df.max())
print(df.idxmax())
print(df.min())
print(df.idxmin())


Category                            Personal Care
name                      iD Wheat Lachha Paratha
mrp                                        260000
discountPercent                                51
availableQuantity                               6
discountedSellingPrice                     139900
weightInGms                                 10000
outOfStock                                   True
quantity                                     1500
dtype: object
Category                  2753
name                      1382
mrp                        517
discountPercent           2608
availableQuantity           53
discountedSellingPrice     517
weightInGms                108
outOfStock                  87
quantity                  2817
dtype: int64
Category                                                          Beverages
name                      "Godrej Aer Power Pocket - Long Lasting Bathro...
mrp                                                                       0
discountPercent 

### Removing duplicated across the dataframe
#### Something noteworthy here is that few categories get removed as they are pure duplicated of another category, this reduces the error caused by dulicate values across multiple different columns


In [4]:
dfN = df.drop_duplicates(subset='name').copy()
dfN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1681 entries, 0 to 3728
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Category                1681 non-null   object
 1   name                    1681 non-null   object
 2   mrp                     1681 non-null   int64 
 3   discountPercent         1681 non-null   int64 
 4   availableQuantity       1681 non-null   int64 
 5   discountedSellingPrice  1681 non-null   int64 
 6   weightInGms             1681 non-null   int64 
 7   outOfStock              1681 non-null   bool  
 8   quantity                1681 non-null   int64 
dtypes: bool(1), int64(6), object(2)
memory usage: 119.8+ KB


#### The more the unique values, less it becomes easy to classify, hence labels produces multiple columns that have less unqiue values thereby providing better analysis across similar groups


In [5]:
labels = ['Category', 'discountPercent', 'availableQuantity']

In [6]:
for value in labels:
    print(f'Unique in {value}: ', df[f'{value}'].sort_values(ascending=True).unique(), 'with length of : ', len(df[f'{value}'].unique()))
    print()

Unique in Category:  ['Beverages' 'Biscuits' 'Chocolates & Candies' 'Cooking Essentials'
 'Dairy, Bread & Batter' 'Fruits & Vegetables' 'Health & Hygiene'
 'Home & Cleaning' 'Ice Cream & Desserts' 'Meats, Fish & Eggs' 'Munchies'
 'Paan Corner' 'Packaged Food' 'Personal Care'] with length of :  14

Unique in discountPercent:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 35 40 43 45 46 49 50 51] with length of :  42

Unique in availableQuantity:  [0 1 2 3 4 5 6] with length of :  7



### Plotting Histogram for representation of quantites that are present across each categories of the dataset


In [7]:
fig = px.histogram(dfN, x='Category', color = 'Category', color_discrete_sequence=colors, template='plotly_dark',height=750)
fig.show()

### Pie Chart Representation of OUT OF STOCK
#### This has been done for each category as it gives a clear direction as to which category has the highest out of stock ratio to another and helps the user understand whether an item from that category will be later available or not
#### Highest percent - Biscuits (maybe due to higher consumption)
#### Lowest percent - Fruits and Vegetables (maybe due to lower consumption OR higher customer demands)


In [8]:
uniqueCategories = dfN['Category'].unique()
n, m = 1, 1
colorPie = ['#ff6361','#ffa600']
subFig1 = mks(rows=1, cols=5, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])
subFig2 = mks(rows=1, cols=4, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])

# for value in uniqueCategories:
print(f'The Probability of being out of stock for  is : ')
for value in uniqueCategories:

    if n<=5:
        dfTemp = dfN[dfN['Category'] == f'{value}']
        # figPie = px.pie(dfTemp, names='outOfStock', color ='outOfStock', color_discrete_sequence=colors[n:n+2], template='plotly_dark', width=500, title=f'{value}')
        countBool = dfTemp['outOfStock'].value_counts().tolist()

        subFig1.add_trace(
            go.Pie(values=countBool, labels=['True', 'False'], title=f'{value}'),
            row=1, col=n).update_traces(hoverinfo='label+value', textinfo='percent', marker=dict(colors=colorPie))
        n+=1
        probs = round(((countBool[1]/countBool[0])*100), 5)
        cons.print(f'{value:<25} : {probs:<1} %')
    else:
        dfTemp = dfN[dfN['Category'] == f'{value}']
        countBool = dfTemp['outOfStock'].value_counts().tolist()
        probs = round(((countBool[1]/countBool[0])*100), 5)
        cons.print(f'{value:<25} : {probs:<1} %')
        subFig2.add_trace(
            go.Pie(values=countBool, labels=['True', 'False'], title=f'{value}'),
            row=1, col=m).update_traces(hoverinfo='label+value', textinfo='percent', marker=dict(colors=colorPie))
        m+=1
subFig1.show()
subFig2.show()

# f.show()

The Probability of being out of stock for  is : 


Fruits & Vegetables       : 4.70588 %

Cooking Essentials        : 13.80952 %

Dairy, Bread & Batter     : 30.12048 %

Packaged Food             : 9.96169 %

Meats, Fish & Eggs        : 28.0 %

Biscuits                  : 35.71429 %

Personal Care             : 6.16438 %

Home & Cleaning           : 9.46746 %

Health & Hygiene          : 5.35714 %

Overall chart, can be uncommented to visualize.

In [9]:
 # fig = px.pie(dfN, names='outOfStock', color ='outOfStock', color_discrete_sequence=colors, template='plotly_dark')
# countBool = dfN['outOfStock'].value_counts().tolist()
# print(countBool)
# print(f'The Probability of being out of stock for overall is : ', round(((countBool[1]/countBool[0])*100), 5), '%')
# fig.show()

## Available Quantity Pie Chart
### Highest of `6` meaning that Zepto has a lot of customers and hence keeps items in more quantities for bulk buy, followed by 3 and 0 meaning Zepto consumers keep buying at a regular basis

In [10]:
fig = px.pie(dfN, names='availableQuantity', color = 'availableQuantity', color_discrete_sequence=colors, template='plotly_dark').update_traces(hoverinfo='label+value', textinfo='percent', marker=dict(colors=colorPie))
fig.show()

## Discount Percent Pie Chart
### Highest being `0%`. This may occur due to Zepto consumers buying products at 0% discount meaning the company is able to profit from that but can be does provide `10%` discount as per sale or consumer need.

In [11]:
fig = px.pie(dfN,names='discountPercent', color = 'discountPercent', color_discrete_sequence=colors, template='plotly_dark').update_traces(textposition='inside').update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

## MRP to Discounted Selling Price Line Chart
### This shows a line comparison of the MRP to the Discounted Selling Price, for each category, it helps in analyzing the if the particular product is off by a massive or small margin. The more the gap, may mean that the product is either not in requirement at the time or maybe profit on it isn't much of an interest to Zepto

In [12]:

subFig = mks(rows=1, cols=2)
for value in uniqueCategories:
    dfT1 = df[df['Category'] == f'{value}']
    dfT2 = dfT1.sort_values(by='quantity', ascending = False)
    fig1 = go.Line(y = dfT2['mrp'],line=dict(color='#bc5090', width=4),opacity=0.8)
    fig2 = go.Line(y = dfT2['discountedSellingPrice'] ,line=dict(color='#ffa600', width=4), opacity=0.8)
    print(f'{value} with count - ', dfT2['quantity'].count())
    subFig.add_trace(fig1)
    subFig.add_trace(fig2)
    subFig.update_traces(mode="markers+lines").update_layout(width=2800,autosize=True, hovermode='x').show()
    subFig.update_traces(visible=False)




Fruits & Vegetables with count -  93


C:\Users\Tanishq\PycharmProjects\zepto-inv-dataset\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




Cooking Essentials with count -  514


Dairy, Bread & Batter with count -  129


Packaged Food with count -  388


Meats, Fish & Eggs with count -  63


Biscuits with count -  147


Personal Care with count -  344


Home & Cleaning with count -  194


Health & Hygiene with count -  97


## 3D Scatter plots for comparison of the availableQunatites with multiple prices
### These plots look into each category and checks if the availableQuantities has any influnce to the increasing prices of MRP and it can be observed that few category do show an indication while others have no relation

In [13]:
for value in uniqueCategories:
    dfT = dfN[dfN['Category'] == f'{value}']
    fig3dScatter = px.scatter_3d(dfT, x='quantity',y='discountedSellingPrice',z='mrp', color='availableQuantity', title=f'{value}', color_continuous_scale='oryel')
    fig3dScatter.update_layout(height=750).update_traces(marker=dict(size=8, symbol="diamond", line=dict(width=2, color="DarkSlateGrey")),selector=dict(mode="markers")).show()

## Zepto OUT OF STOCK Item Refill Analysis
### Finally we come to the end which shows which item from each category is out of stock and should be refilled on the basis of the quantity and the weight.

In [14]:
cons.print('The following shows the items in each categories that are in highest quantities, yet are out of stock and has to be refilled!', style='light_green')
for value in uniqueCategories:
    dfTemp = dfN[dfN['Category'] == f'{value}'].drop(columns='Category')
    # cons.print(f'{value}\n\n{dfTemp['availableQuantity'].value_counts()}', style='medium_purple1')
    dfTemp['overallWeight'] = dfTemp['weightInGms']*dfTemp['quantity']
    dfTemp2 = dfTemp[dfTemp['outOfStock']==True].sort_values(by=['overallWeight','discountPercent'],ascending=False).head(20).copy()
    cons.print(f'\n\n{value}\n\n{dfTemp2[['name','weightInGms','quantity','discountPercent']]}')


The following shows the items in each categories that are in highest quantities, yet are out of stock and has to be
refilled!

Fruits & Vegetables

                             name  weightInGms  quantity  discountPercent
90  Safal Frozen Mixed Vegetables          500       500                5
89                  Moong Sprouts          200       200               17
91              Safal Frozen Corn          200       200                4
92      Safal Frozen - Sweet Corn         1000         1                5

Cooking Essentials

                                                  name  weightInGms  quantity  \
571                   Ching's Secret Veg Hakka Noodles          600       600   
589                                   Top Ramen Masala          560       560   
593              Chef's Basket Durum Wheat Penne Pasta          500       500   
599            Sugarlite 50% Less Calories Sugar Pouch          500       500   
573                 Everest Kashmiri Lal Chilli Powder          500       500   
583                        Popular Essentials Sabudana          500       500   
569  Popular Essentials Black Brown Chana (Un-Polis...          500       500   
591    Popular Essentials Buckwheat Flour (Kuttu Atta)          500       500   
546                                   Satyam Poha Jada          500       500   
561                             Satyam Thin Poha Patla          500       500   
554                                  Tata Sampann Poha          500       500   
564                                   Satyam Singdana           500       500   
565                         Tata Sampann Moong Chilka           500       500   
566         Tata Sampann High Protien Toor Dal - Split          500       500   
563        Tata Sampann High Protien Chana Dal - Split          500       500   
584                     Popular Essentials Brown Sugar          500       500   
559         Tata Sampann Unpolished Toor Dal/Arhar Dal         1000       200   
597                   Zorabian Chicken Kheema Parathas          300       300   
551            Maggi 2-Minute Instant Noodles - Masala          280       280   
604           Maggi Special Masala Spicy Yummy Noodles          280       280   

     discountPercent  
571                0  
589                9  
593               50  
599               40  
573               10  
583               10  
569                9  
591                9  
546                8  
561                7  
554                6  
564                6  
565                6  
566                2  
563                0  
584                0  
559                1  
597                0  
551                0  
604                0

Dairy, Bread & Batter

                                                   name  weightInGms  \
1231                      Nestle A+ Slim Dahi - Low Fat          400   
1226                    The Bake Shop Whole Wheat Bread          400   
1237                    The Bake Shop Multi Grain Bread          400   
1240  The Whole Truth Peanut Butter - Unsweetened & ...          325   
1232                      Britannia Cheese Garlic Bread          300   
1225                             Amul Lactose Free Milk          250   
1235                              Britannia Atta Kulcha          250   
1249                        RRO Mozzarella Block Cheese          200   
1239         Go Cheese Pizza Blend Mozzarella + Cheddar          200   
1222                           Amul Malai Paneer Fresh           200   
1241                    Amul Cheese Spread Spicy Garlic          200   
1229                     Amul Pizza Cheese - Mozzarella          200   
1233                 The Bake Shop Whole Wheat Ladi Pav          200   
1234                     Amul Cheese Spread Yummy Plain          200   
1247                         Prabhat Malai Paneer Pouch          200   
1246                   Pillsbury Pancake Mix - Funfetti          162   
1230               The Bake Shop Pizza Base Whole Wheat          150   
1243                 D'lecta Shredded Mozzarella Cheese          140   
1242                          The Bake Shop Burger Buns          100   
1236                                 Suguna Nourish Egg          696   

      quantity  discountPercent  
1231       400                3  
1226       400                0  
1237       400                0  
1240       325               20  
1232       300                0  
1225       250                4  
1235       250                0  
1249       200               50  
1239       200                5  
1222       200                3  
1241       200                3  
1229       200                1  
1233       200                0  
1234       200                0  
1247       200                0  
1246       162               11  
1230       150                0  
1243       140                5  
1242       100                0  
1236        12                0

Packaged Food

                                                 name  weightInGms  quantity  \
1753    Del Monte Original Blend Tomato Ketchup Pouch          950       950   
1744   Kelloggs Corn Flakes With Real Strawberry Pure          575       575   
1755                           Del Monte Eggless Mayo          500       500   
1725                          Kissan Mixed Fruit Jam           500       500   
1762                               Yummiez Green Peas          500       500   
1766                    Del Monte Pitted Green Olives          450       450   
1757                    Mother's Recipe Mixed Pickle           400       400   
1764                      Yummiez American Sweet Corn          400       400   
1759                  Hershey's Cocoa + Almond Spread          350       350   
1756            ITC Master Chef Cheesy Corn Triangles          320       320   
1741       Veeba Favourites Eggless Burger Mayonnaise          250       250   
1765  Dr Oetker Cucumber & Carrot Sandwich Spread Veg          250       250   
1728          Mother's Recipe Ginger And Garlic Paste          200       200   
1726                                Lijjat Urad Papad          200       200   
1738                               Lijjat Moong Papad          200       200   
1761                         Del Monte Schezwan Sauce          190       190   
1754                       Del Monte Natural Vinegar           180       180   
1724                      Kellogg's Chocos Combo Pack          156       156   
1748                Dabur Hommade - Imli Sauce Saunth          100       100   
1723          Kurkure Green Chutney, Rajasthani Style           90        90   

      discountPercent  
1753                7  
1744               10  
1755                6  
1725                0  
1762                0  
1766                0  
1757                4  
1764                0  
1759                9  
1756                5  
1741                8  
1765                0  
1728               24  
1726                5  
1738                5  
1761                0  
1754                3  
1724                6  
1748               10  
1723                0

Meats, Fish & Eggs

                                            name  weightInGms  quantity  \
2598        Fresh Chicken Mince - Kheema (Halal)          500       500   
2595  Fresh Chicken Curry Cut - Skinless (Halal)          500       500   
2605                      Zorabian Chicken Cubes          500       500   
2599    Godrej Real Good Chicken Boneless Cubes           450       450   
2596           Zorabian Chicken Breast Boneless           450       450   
2602                      Zorabian Chicken Mince          450       450   
2603              Zorabian Chicken Leg Boneless           450       450   

      discountPercent  
2598                5  
2595                0  
2605                0  
2599                9  
2596                0  
2602                0  
2603                0

Biscuits

                                                   name  weightInGms  \
2724            Britannia Marie Gold (Buy 4 Get 1 Free)          600   
2727                  Britannia Vita Marie Gold Biscuit          300   
2737  Britannia Nutri Choice Cracker Sugar Free Biscuit          300   
2738  Cadbury Oreo Chocolatey Sandwich Biscuits Fami...          300   
2751                          Patanjali Atta Doodh Rusk          300   
2721                 Dukes Waffy Chocolate Wafer Rolls           250   
2712               Britannia Marie Gold Biscuits Packet          250   
2749                Parle Monaco Sixer Salted Biscuits           200   
2719                 Britannia Milk Bikis Cream Biscuit          200   
2723                       Britannia Marie Gold Biscuit          200   
2731   Britannia Nutrichoice 5 Grain Digestive Biscuits          200   
2742                          The Bake Shop Jeera Khari          200   
2750                Sunfeast Dark Fantasy Bourbon Bliss          150   
2730                              Sunfeast Nice Biscuit          150   
2720         Britannia Bourbon Chocolate Cream Biscuits          150   
2729                    Parle Fab! Jam-In Cream Filled           150   
2718                        Britannia Nice Time Biscuit          150   
2728                          The Bake Shop Maska Khari          200   
2732                    Cadbury Oreo Strawberry Bicuits          120   
2743  Cadbury Oreo Creme Biscuit - Vanilla, Original...          120   

      quantity  discountPercent  
2724       600                0  
2727       300                0  
2737       300                0  
2738       300                0  
2751       300                0  
2721       250               50  
2712       250                0  
2749       200               10  
2719       200                0  
2723       200                0  
2731       200                0  
2742       200                0  
2750       150               17  
2730       150               12  
2720       150                2  
2729       150                2  
2718       150                0  
2728       100                6  
2732       120                3  
2743       120                3

Personal Care

                                                   name  weightInGms  \
3076             Dettol Original Liquid Handwash Refill          750   
3080           Parachute Advansed Gold Coconut Hair Oil          400   
3095            Godrej Cinthol Lime Refreshing Deo Soap          400   
3077                    Parachute 100% Pure Coconut Oil          300   
3093                 Bajaj Almond Drops Hair Oil Bottle          200   
3086                  Godrej No.1 Bathing Soap, Jasmine          400   
3079                  Dabur Gulabari Premium Rose Water          120   
3094       Whisper Bindazzz Night Sanitary Pads XL Plus          870   
3092                        Bajaj Almond Drops Hair Oil          100   
3091       Clean & Clear Morning Energy Lemon Face Wash          100   
3083           Sensodyne Sensitive Fresh Gel Toothpaste           75   
3085                       Whisper Choice Sanitary Pad           406   
3078  Stayfree Secure Cottony Soft Cover Sanitary Pa...          406   
3089  Stayfree Secure Dry Cover Extra Large Sanitary...          406   
3084          Whisper Bindazzz Nights XL + Sanitary Pad          406   
3090  Oxylife Natural Radiance 5 Creme Bleach - With...           27   
3096  MamyPoko Pants Standard Diapers, Extra Large (...           26   
3087                Gillette Venus Close & Clean Razor            58   

      quantity  discountPercent  
3076       750                3  
3080       400                9  
3095       400                0  
3077       300                0  
3093       200                7  
3086       100                0  
3079       120                2  
3094        15                0  
3092       100               10  
3091       100                9  
3083        75                7  
3085         7                9  
3078         7                3  
3089         7                2  
3084         7                0  
3090        27                9  
3096        26                7  
3087         1                7

Home & Cleaning

                                                   name  weightInGms  \
3629  Comfort After Wash Fabric Conditioner Lily Fresh           860   
3628                      Rin Fabric Whitener Detergent          500   
3616                                   Vim Dishwash Bar          300   
3617      Dettol Effective Protection Antiseptic Liquid          250   
3621                                  Rin Detergent Bar          140   
3633                               Origami Plates Plain          100   
3622                    Odomos Mosquito Repellent Spray          100   
3630       Ujala Superme For Super Whiteness Of Clothes           75   
3618     Godrej Good Night Gold Flash Liquid Vapouriser           60   
3620              Surf Excel Easy Wash Detergent Powder         1000   
3619                               Rin Detergent Powder         1000   
3624  Scotch-Brite Scrub Sponge Ideal For Dishwash L...          116   
3632                         Scotch BOPP Packaging Tape           58   
3623                    Chakaachak Super Steel Scrubber           58   
3625                                 Gala No Dust Broom           58   
3626                   Chakaachak Eazo Grass Broom New            58   

      quantity  discountPercent  
3629       860                9  
3628       500               10  
3616       300                4  
3617       250                0  
3621       140                0  
3633       100               10  
3622       100                8  
3630        75                0  
3618        60                0  
3620         1                3  
3619         1                2  
3624         2                9  
3632         1               15  
3623         1                8  
3625         1                7  
3626         1                2

Health & Hygiene

                                                   name  weightInGms  \
3728  Dabur Honitus Herbal Cough Remedy Ayurvedic Syrup          100   
3719                   Epigamia Fruit Yogurt Strawberry           75   
3722   Eno Cooling Sachets - Cool Mint, 6 Pieces Carton            5   

      quantity  discountPercent  
3728       100               15  
3719        75               50  
3722         5               11